In [29]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import folium
from folium.plugins import MarkerCluster

In [21]:
url = 'https://www.starbucks.co.id/store-locator'

In [27]:
# Set up the Selenium webdriver
driver = webdriver.Chrome()

# Navigate to the URL
driver.get(url)

# Add a delay of 10 seconds (if needed)
time.sleep(5)

# Get the page source after JavaScript execution
page_source = driver.page_source

# Close the Selenium WebDriver
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Initialize lists to store data
names = []
addresses = []
hours = []
distances = []

# Initialize lists to store data
data = []

# Find all the card elements
card_elements = soup.find_all('div', class_='card')

for card in card_elements:
    # Extract information from each card
    name = card.find('div', class_='text_head').find('h3').get_text(strip=True)
    address = card.find('div', class_='card_left').find('p').get_text(strip=True)
    hour_text = card.find('div', class_='card_left').find('h4').get_text(strip=True)
    distance = card.find('a', class_='card_right').find('strong').get_text(strip=True)
    map_link = card.find('a', class_='card_right')['href']

    # Append the extracted data to the list as a dictionary
    data.append({'Name': name, 'Address': address, 'Hours': hour_text, 'Distance': distance, 'Map_Link': map_link})

# Create a DataFrame
df = pd.DataFrame(data)

In [28]:
df

,Name,Address,Hours,Distance,Map_Link
0,Tebet Indraya Square (TIS Square),"Starbucks TIS Square, Jl. Letjen M.T. Haryono ...",Mon - Sun : 07.00 - 22.00,0.6Km,https://goo.gl/maps/8XXAkKj8bV4aksYg9
1,La Avenue,Starbucks Lávenue Jl. Raya Pasar Minggu Kav 1...,Mon - Fri : 06.30 - 21.00Sat - Sun : 09.00 - 1...,0.8Km,https://goo.gl/maps/caBD53JBmdZiSa9x7
2,Kalibata City Square,"Starbucks Kalibata City Square, Ground Floors,...",Mon - Sun : 08.30 - 22.00,1.1Km,https://goo.gl/maps/STpnaba5wXcp5ujd6
3,Menara Bidakara,"Starbucks Menara Bidakara, Building A, Jl. Jen...",Mon - Sat : 06.00 - 21.00Sun : 09.00 - 17.00,1.2Km,https://goo.gl/maps/VqrGtSUdhrYQnCCeA
4,Supomo Tebet,Starbucks Supomo Tebet Jalan Prof Dr. Supomo N...,Mon - Thu : 06.00 - 23.00Fri - Sat : 06.00 - 2...,1.7Km,https://goo.gl/maps/pScCPC8Hrk37nuRC8
...,...,...,...,...,...
343,Buah Batu Bandung,Buah Batu Bandung Jalan Buah Batu No. 159 Tura...,Mon - Sun : 07.00 - 22.30,115Km,https://goo.gl/maps/AndGC2khJiXYYos57
344,Trans Studio Mall,"Starbucks Trans Studio Mall, Ground Floors, Jl...",Sun - Thu : 08.00 - 22.30Fri - Sat : 08.00 - 2...,115Km,https://goo.gl/maps/hQEmLtd9PAMbJwsp8
345,Raya Jatinangor,"Starbucks Jatinangor, Jl. Raya Jatinangor No.1...",Sun - Thu : 07.00 - 22.00Fri - Sat : 07.00 - 2...,127.1Km,https://goo.gl/maps/CNAuAkBoBDi6Dyka8
346,Ahmad Yani Garut,"Jl. Ahmad Yani No.3, Paminggir Kec Garut 44118",Sun - Thu : 07.00 - 22.00Fri - Sat : 07.00 - 2...,158.1Km,N


In [90]:
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import InvalidArgumentException
import re

In [101]:
def get_coordinates(link):
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Jalankan browser tanpa tampilan GUI
    try:
        # Ganti path dengan path ke file chromedriver.exe di komputer Anda
        driver = webdriver.Chrome()
        driver.get(link)

        # Tunggu sebentar untuk memastikan halaman sepenuhnya dimuat
        driver.implicitly_wait(10)

        # Dapatkan URL saat ini setelah halaman sepenuhnya dimuat
        current_url = driver.current_url

        # Parsing URL untuk mendapatkan koordinat
        match = re.search(r'@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
        if match:
            return float(match.group(1)), float(match.group(2))
        else:
            return "Location not valid"
    except InvalidArgumentException as e:
        # Tangkap exception jika link tidak valid
        print(f"Invalid argument exception: {e}")
        return "Location not valid"
    except Exception as e:
        # Tangkap semua exception yang mungkin terjadi
        print(f"Error: {e}")
        return "Location not valid"
    finally:
        # Tutup browser setelah selesai
        driver.quit()

### Check Sample Row

In [88]:
df2 = df.copy()

In [102]:
# Daftar indeks yang ingin diperbarui
indeks_diperbarui = [346, 347]

# Menerapkan fungsi get_coordinates pada setiap link dan menyimpan hasilnya dalam kolom X dan Y
df2.loc[indeks_diperbarui, ['X', 'Y']] = df2.loc[indeks_diperbarui, 'Map_Link'].apply(get_coordinates)

Invalid argument exception: Message: invalid argument
  (Session info: chrome=121.0.6167.85)
Stacktrace:
	GetHandleVerifier [0x00007FF68A1C5E42+3538674]
	(No symbol) [0x00007FF689DE4C02]
	(No symbol) [0x00007FF689C9599D]
	(No symbol) [0x00007FF689C833C6]
	(No symbol) [0x00007FF689C813F1]
	(No symbol) [0x00007FF689C81C8C]
	(No symbol) [0x00007FF689C98179]
	(No symbol) [0x00007FF689D1C757]
	(No symbol) [0x00007FF689CFF0AA]
	(No symbol) [0x00007FF689D1BDE3]
	(No symbol) [0x00007FF689CFEE53]
	(No symbol) [0x00007FF689CCF514]
	(No symbol) [0x00007FF689CD0631]
	GetHandleVerifier [0x00007FF68A1F6CAD+3738973]
	GetHandleVerifier [0x00007FF68A24C506+4089270]
	GetHandleVerifier [0x00007FF68A244823+4057299]
	GetHandleVerifier [0x00007FF689F15C49+720121]
	(No symbol) [0x00007FF689DF126F]
	(No symbol) [0x00007FF689DEC304]
	(No symbol) [0x00007FF689DEC432]
	(No symbol) [0x00007FF689DDBD04]
	BaseThreadInitThunk [0x00007FFC8D32257D+29]
	RtlUserThreadStart [0x00007FFC8E26AA58+40]



### Running All Location

In [ ]:
df2['X'], df2['Y'] = zip(*df2['Map_Link'].apply(get_coordinates))

In [103]:
df2

,Name,Address,Hours,Distance,Map_Link,X,Y
0,Tebet Indraya Square (TIS Square),"Starbucks TIS Square, Jl. Letjen M.T. Haryono ...",Mon - Sun : 07.00 - 22.00,0.6Km,https://goo.gl/maps/8XXAkKj8bV4aksYg9,NaN,NaN
1,La Avenue,Starbucks Lávenue Jl. Raya Pasar Minggu Kav 1...,Mon - Fri : 06.30 - 21.00Sat - Sun : 09.00 - 1...,0.8Km,https://goo.gl/maps/caBD53JBmdZiSa9x7,NaN,NaN
2,Kalibata City Square,"Starbucks Kalibata City Square, Ground Floors,...",Mon - Sun : 08.30 - 22.00,1.1Km,https://goo.gl/maps/STpnaba5wXcp5ujd6,NaN,NaN
3,Menara Bidakara,"Starbucks Menara Bidakara, Building A, Jl. Jen...",Mon - Sat : 06.00 - 21.00Sun : 09.00 - 17.00,1.2Km,https://goo.gl/maps/VqrGtSUdhrYQnCCeA,NaN,NaN
4,Supomo Tebet,Starbucks Supomo Tebet Jalan Prof Dr. Supomo N...,Mon - Thu : 06.00 - 23.00Fri - Sat : 06.00 - 2...,1.7Km,https://goo.gl/maps/pScCPC8Hrk37nuRC8,NaN,NaN
...,...,...,...,...,...,...,...
343,Buah Batu Bandung,Buah Batu Bandung Jalan Buah Batu No. 159 Tura...,Mon - Sun : 07.00 - 22.30,115Km,https://goo.gl/maps/AndGC2khJiXYYos57,NaN,NaN
344,Trans Studio Mall,"Starbucks Trans Studio Mall, Ground Floors, Jl...",Sun - Thu : 08.00 - 22.30Fri - Sat : 08.00 - 2...,115Km,https://goo.gl/maps/hQEmLtd9PAMbJwsp8,NaN,NaN
345,Raya Jatinangor,"Starbucks Jatinangor, Jl. Raya Jatinangor No.1...",Sun - Thu : 07.00 - 22.00Fri - Sat : 07.00 - 2...,127.1Km,https://goo.gl/maps/CNAuAkBoBDi6Dyka8,-6.934299,107.768327
346,Ahmad Yani Garut,"Jl. Ahmad Yani No.3, Paminggir Kec Garut 44118",Sun - Thu : 07.00 - 22.00Fri - Sat : 07.00 - 2...,158.1Km,N,Location not valid,Location not valid


In [69]:
# Create a Folium map centered around Jakarta
map_center = [-6.2088, 106.8456]  # Jakarta's latitude and longitude
starbucks_map = folium.Map(location=map_center, zoom_start=10)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(starbucks_map)

# Add markers for each location to the MarkerCluster layer
for index, row in df2.iterrows():
    folium.Marker([row['X'], row['Y']], popup=row['Name']).add_to(marker_cluster)

In [70]:
starbucks_map